In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Any, Dict, Optional

import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import EnvSpec
from gymnasium.utils import seeding
from gymnasium.envs.registration import register
import numpy as np
from openrl.envs.common import make

In [44]:
def load_data(time_steps,excel_file = 'data/train.xlsx'):
    import pandas as pd
    data1 = pd.read_excel(excel_file, sheet_name='Sheet1')
    data1['date'] = pd.to_datetime(data1['date'])
    data2 = pd.read_excel(excel_file, sheet_name='Sheet2')
    data2['date'] = pd.to_datetime(data2['date'])
    merged_data = pd.merge_asof(data2, data1, on='date', tolerance=pd.Timedelta('1H'))
    
    merged_data.keys()
    selected_columns = ['pri_supp_t','pri_back_t', 'pri_flow', 'sec_supp_t', 'sec_back_t', 'sec_flow', 'outdoor', 'irradiance']
    data_subset = merged_data[selected_columns]
    data_subset.dropna(inplace=True)
    X = data_subset.values
    X_sequence = []
    for i in range(len(data_subset) - time_steps):
        X_sequence.append(X[i:i+time_steps])
    X_sequence = np.array(X_sequence, dtype=np.float32)
    return X_sequence    

In [45]:
def load_test_data(time_steps):
    df1, df2 = load_data(time_steps, 'data/train.xlsx'), load_data(time_steps, 'data/test.xlsx')
time_steps=64
df1, df2 = load_data(time_steps, 'data/train.xlsx'), load_data(time_steps, 'data/test.xlsx') 

/tmp/ipykernel_54827/3787778844.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.dropna(inplace=True)
/tmp/ipykernel_54827/3787778844.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.dropna(inplace=True)


In [51]:
excel_file='data/test.xlsx'
data1 = pd.read_excel(excel_file, sheet_name='Sheet1')
data1['date'] = pd.to_datetime(data1['date'])
data2 = pd.read_excel(excel_file, sheet_name='Sheet2')
data2['date'] = pd.to_datetime(data2['date'])
merged_data = pd.merge_asof(data2, data1, on='date', tolerance=pd.Timedelta('1H'))
    
merged_data.keys()
selected_columns = ['pri_supp_t','pri_back_t', 'pri_flow', 'sec_supp_t', 'sec_back_t', 'sec_flow', 'outdoor', 'irradiance']
data_subset = merged_data[selected_columns]
data_subset.dropna(inplace=True)
df2.to_excel('test_1H.xlsx')

AttributeError: 'numpy.ndarray' object has no attribute 'to_excel'

In [50]:
import pandas as pd

# 读取 CSV 文件
csv_file = 'output.csv'  # 请替换为您的 CSV 文件名
data = pd.read_csv(csv_file)

# 将数据写入 Excel 文件
xlsx_file = 'output.xlsx'  # 指定输出的 Excel 文件名
data.to_excel(xlsx_file, index=False)

print(f"已将 {csv_file} 转换为 {xlsx_file}")


已将 output.csv 转换为 output.xlsx


In [30]:
data_subset.shape

(791, 8)

In [110]:
from src.mlpmixer import MLPMixer
import json

In [189]:
import torch
def load_data(time_steps):
    import pandas as pd
    excel_file = 'data/train.xlsx'
    data1 = pd.read_excel(excel_file, sheet_name='Sheet1')
    data1['date'] = pd.to_datetime(data1['date'])
    data2 = pd.read_excel(excel_file, sheet_name='Sheet2')
    data2['date'] = pd.to_datetime(data2['date'])
    merged_data = pd.merge_asof(data1, data2, on='date', tolerance=pd.Timedelta('1H'))
    merged_data.keys()
    selected_columns = ['pri_supp_t','pri_back_t', 'pri_flow', 'sec_supp_t', 'sec_back_t', 'sec_flow', 'outdoor', 'irradiance']
    data_subset = merged_data[selected_columns]
    data_subset.dropna(inplace=True)
    X = data_subset.values
    X_sequence = []
    tmp=[]
    for i in range(len(data_subset) - time_steps):
        tmp.append(X[i:i+time_steps])
        X_sequence.append((np.concatenate(X[i:i+time_steps])))
    X_sequence = np.array(X_sequence, dtype=np.float32)
    return X_sequence, np.array(tmp, dtype=np.float32 )

def yield_data(data):
    for x in data:
        yield x

def load_config(path):
    return json.load(path, 'r')

class IdentityEnv(gym.Env):
    spec = EnvSpec("IdentityEnv")
    def __init__(self,time_steps,path_to_model):
        self.dim = 2
        self.observation_space = spaces.Discrete(1)
        self.action_space = spaces.Discrete(self.dim)
        self.time_steps = time_steps
        self.current_step = 0
        datas, tmp = load_data(time_steps)
        self.ep_length = len(datas)
        self.yield_data = yield_data(datas)
        self.yield_predict = yield_data(tmp)
        #config for openrl
        self.agent_num=1
        self.parallel_env_num = 1      
        self.env_name = 'water'  
        # Get action space
        self.action_space = gym.spaces.Box(low=-2, high=2, shape=(1,))
        # Get observation space
        max_value = []
        min_value = []
        for i in range(8):
            max_value.append(round(data[:,:,i].max())+1)
            min_value.append(round(data[:,:,i].min())-1)
        self.observation_space = gym.spaces.Box(low=np.array([min_value]*time_steps), high=np.array([max_value]*time_steps),shape=(time_steps,8))
        #import the simulator
        config = json.load(open('/home/yuleliu/art/model_weight/model_config.json','r'))
        self.model = MLPMixer(**config,batched=False)
        self.model.to(self.model.device)
        self.model.load_state_dict(torch.load(path_to_model))

    def reset(
            self,
            *,
            seed: Optional[int] = None,
            options: Optional[Dict[str, Any]] = None,
    ):
        if seed is not None:
            self.seed(seed)
        self.current_step = 0
        datas, tmp = load_data(self.time_steps)
        self.yield_data = yield_data(datas)
        self.yield_predict = yield_data(tmp)
        return next(self.yield_data), {}
    
    def get_tem(self, x):
        with torch.no_grad():
            self.model.eval()
            predicted = self.model(torch.from_numpy(next(env.yield_data)))
        return float(predicted)

    def step(self, action) :
        state = next(self.yield_data)
        tem_origin = state[-1,3]
        state[-1,3] = state[-1,3]+action
        tem_in = self.get_tem(state)
        reward = 0
        if 20<=tem_in<=24:
            reward+=1
            if state[-1,3]<tem_origin:
                reward+=0.5
            else:
                reward-=0.5
        else:
            reward-=1
        # print(state[-1,3],tem_origin,tem_in)
        self.current_step += 1
        done = self.current_step >= self.ep_length
        return state, reward, done, {}
    
    def render(self, mode: str = "human") -> None:
        pass

    def seed(self, seed: Optional[int] = None) -> None:
        if seed is not None:
            self._np_random, seed = seeding.np_random(seed)
            
    def close(self):
        pass
    
path_to_model = '/home/yuleliu/art/model_weight/model_step64_dropout_0.1_dim_256_layer_16_test_0.274868298345625.pth'
env = IdentityEnv(64,path_to_model)

/tmp/ipykernel_102765/3925532614.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.dropna(inplace=True)


In [190]:
env.reset()[0].shape

/tmp/ipykernel_102765/3925532614.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.dropna(inplace=True)


(512,)

In [181]:
[1,2]*3

[1, 2, 1, 2, 1, 2]

In [159]:
from openrl.modules.common.ppo_net import PPONet as Net
from openrl.runners.common.ppo_agent import PPOAgent as Agent
agent = Agent(Net(env,device='cuda')) # 直接传入该环境即可
agent.train(5000) # 开始训练！


/tmp/ipykernel_102765/2606623039.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.dropna(inplace=True)
/tmp/ipykernel_102765/2606623039.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset.dropna(inplace=True)


[01/03/24 17:17:57] INFO     2024-01-03 17:17:57,682 [INFO] Episode: 0/25                             ]8;id=441001;file:///home/yuleliu/miniconda3/lib/python3.10/site-packages/openrl/utils/logger.py\logger.py]8;;\:]8;id=42450;file:///home/yuleliu/miniconda3/lib/python3.10/site-packages/openrl/utils/logger.py#164\164]8;;\

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x8 and 64x64)

In [191]:
3e6


3000000.0